In [9]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, TruncatedSVD, NMF
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import ShuffleSplit, KFold
import matplotlib.pyplot as plt
from scipy import sparse
from itertools import product
from time import time
import seaborn
import pickle
from collections import Counter
import heapq
import pandas as pd

In [10]:
#load everything
num_ing = 8023
num_items = 8576
num_rec = 178265
recipe_decoder = pickle.load( open('data-cleaned/recipe_decoder.pkl', 'rb') )
ingr_decoder   = pickle.load( open('data-cleaned/ingredient_decoder.pkl', 'rb') )
tag_decoder    = pickle.load( open('data-cleaned/tag_decoder.pkl', 'rb') )
R     = sparse.load_npz("data-cleaned/recipes.npz")
Rhat  = sparse.load_npz("data-cleaned/recipes_tfidf.npz")
U     = sparse.load_npz("data-cleaned/user_train.npz")
Uhat  = sparse.load_npz("data-cleaned/user_train_tfidf.npz")
Utest = sparse.load_npz("data-cleaned/user_test.npz")
col_sum = np.load("data-cleaned/recipe_col_sum.npy")
user_test = Utest[:,-1].toarray().flatten().astype('int')
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

In [21]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sklearn.pipeline import Pipeline
import recipeClasses
import userClasses
import importlib
from baseUtils import *

In [31]:
#prepare testing data and splitting data
user_test = Utest[:,-1].toarray().flatten().astype('int')
y = np.zeros((user_test.shape[0], 2), dtype='int')-1
for i in range(len(y)):
    recipes = Utest[i].nonzero()[1][:-1]
    if len(recipes) == 1:
        y[i,0] = recipes
    elif len(recipes) == 2:
        y[i,:] = recipes
    else:
        raise ValueError("Someone reviewed 3 recipes!")

# Urecipe = user_to_recipe(U[user_test], 4)
y = y

In [5]:
Upca = TruncatedSVD(n_components=10).fit_transform(U)
test = Upca[user_test]

In [42]:
importlib.reload(userClasses)

<module 'userClasses' from '/home/contagon/Documents/ml-project/userClasses.py'>

In [44]:
rdr = userClasses.userCluster(n_clusters=100)
rdr.fit(Upca)
rdr.score(test[0:100], y[0:100])

8.006

In [39]:
userClasses.recommend_sum(np.array([]))

array([  2890, 168844, 109210,  51313,  17076])

In [9]:
columns = ["kNN", "NNBall", "KMeans", "GMM", "MinCut", "KMeansNN", "GMMNN", "MinCutNN", ]
rows = ["PCA", "KPCA", "NMF", "LDA"]
d = pd.DataFrame(tuple(), columns=columns, index=rows)
# d.to_pickle("results/user_U_sum.pkl")
# d.to_pickle("results/user_U_fr.pkl")
# d.to_pickle("results/user_U_rf.pkl")
# d.to_pickle("results/user_Uhat_sum.pkl")
# d.to_pickle("results/user_Uhat_fr.pkl")
# d.to_pickle("results/user_Uhat_rf.pkl")
d.to_pickle("results/recipe_R_int.pkl")
d.to_pickle("results/recipe_R_com.pkl")
d.to_pickle("results/recipe_Rhat_int.pkl")
d.to_pickle("results/recipe_Rhat_com.pkl")

In [ ]:
columns = [1, 2, 3, 4, 5]
rows = ["PCA", "KPCA", "NMF", "LDA"]
d = pd.DataFrame(tuple(), columns=columns, index=rows)
d.to_pickle("results/recipe_R_int.pkl")
d.to_pickle("results/recipe_R_com.pkl")
d.to_pickle("results/recipe_Rhat_int.pkl")
d.to_pickle("results/recipe_Rhat_com.pkl")

In [65]:
def addcol(filename): 
    df = pd.read_pickle(filename) 
    for rating in [3,4,5]:
        df["MkNN_"+rating] = np.nan 
    df.to_pickle(filename) 

,kNN,NNBall,KMeans,GMM,MinCut,KMeansNN,GMMNN,MinCutNN,DBSCAN,DBSCANNN
PCA,True,True,True,True,False,True,True,False,False,False
KPCA,True,True,False,True,False,False,False,False,False,False
NMF,True,True,True,True,False,False,False,False,False,False
LDA,True,True,True,True,False,False,False,False,False,False


In [47]:
i=5
j=10
ii = R[i,:-1].nonzero()[1]
ii

array([ 150,  159,  800, 1124, 1645, 1833, 1881, 2206, 5010, 5574, 6276,
       7927, 8156, 8162, 8178, 8188, 8199, 8220, 8307, 8310, 8314, 8320,
       8328, 8409, 8410, 8423], dtype=int32)

In [48]:
jj = R[j,:-1].nonzero()[1]
jj

array([ 298,  339, 1223, 1372, 2728, 2854, 3184, 4836, 5006, 5219, 5407,
       5974, 6906, 6926, 7799, 8027, 8044, 8156, 8165, 8178, 8199, 8320,
       8368, 8384, 8386, 8423, 8456, 8471, 8497, 8518, 8523, 8539],
      dtype=int32)

In [51]:
print(R[i,-1], R[j,-1])

2.0 2.0


In [52]:
intersection(ii, jj)

[8320, 8423, 8199, 8178, 8156]

In [53]:
recipe_score_int(i, np.array([j]))

array([6])

In [54]:
recipe_score_com(i, np.array([j]))

array([-48])

In [55]:
len(ii) + len(jj) - 2*len(intersection(ii,jj)) + int(R[i,-1]!=R[j,-1])

48